<a href="https://colab.research.google.com/github/kkaran0908/Natural-Language-Processing/blob/master/Assignment2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Dataset loading


---





In [0]:
from google.colab import drive
import pandas as pd
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize, RegexpTokenizer
import re
import string
import operator
from nltk.util import ngrams
import numpy as np


nltk.download('punkt')

drive.mount("/content/drive")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
Mounted at /content/drive


In [0]:
# read file from google-drive, colab doesnt keeps uploaded files persistently

data= open(r"drive/My Drive/Colab Notebooks/content/speech.txt", encoding="utf8")
data = data.read()

sent_tokenize_list = nltk.sent_tokenize(data)

In [0]:
#reference: 
sent_list_clean = []
for sent in sent_tokenize_list:
  # remove \n
  temp = re.sub("\n", ' ', sent)

  # remove ’
  temp = re.sub("\’", '', sent)

  # convert to lowercase
  temp = temp.lower()

  # remove unnecessary punctuations
  temp = " ".join("".join([" " if ch in string.punctuation else ch for ch in temp]).split())
  
  #after cleaning up, append to new list
  sent_list_clean.append(temp)

# sent_list_clean
# len(sent_list_clean)

Divide dataset in 80:20 ratio

In [0]:
training_data = sent_list_clean[:int(len(sent_list_clean)*0.8)]
test_data = sent_list_clean[int(len(sent_list_clean)*0.8):]

print (len(training_data), len(test_data))

13194 3299


### Ans 1  - Classical Approach (explanation inline comments)

####Unigram

In [0]:
#In Unigram we only calculate probability of word, as there is no context word

word_dict = {}
total_num_of_words = 0

for sentence in training_data:
  words=word_tokenize(sentence)
  total_num_of_words += len(words)
  for word in words:
    if word in word_dict:
      word_dict[word] += 1
    else:
      word_dict[word] = 1

# print(total_num_of_words)

unigram_prob_dict = {}
for key, val in reversed(sorted(word_dict.items(), key = operator.itemgetter(1))):
  unigram_prob_dict[key] = val / total_num_of_words

print ("Probability values unigram, sorted desc order")
print(unigram_prob_dict)

Probability values unigram, sorted desc order
{'the': 0.03439235267258775, 'to': 0.03255858483201508, 'and': 0.03210198400424197, 'i': 0.02859646797166129, 'a': 0.02132767737469253, 'you': 0.01840395917104856, 'of': 0.01711516651201155, 'we': 0.015892654618296437, 'it': 0.014353467956932231, 'that': 0.013653837656312139, 'have': 0.013602285949950658, 'they': 0.012821645825048238, 'going': 0.012320857820393855, 'in': 0.011864256992620741, 'so': 0.009109922967021638, 'is': 0.009036277672219522, 'but': 0.00829982472419837, 'know': 0.007916869191227372, 'were': 0.007835859366945046, 'people': 0.0077622140721429304, '–': 0.007526549128776163, 'its': 0.00741608118657299, 'be': 0.007150958125285376, 'are': 0.006318766294021475, 'for': 0.006208298351818302, 'not': 0.006038914173773438, 'this': 0.00600209152637238, 'do': 0.005965268878971323, 'our': 0.005891623584169207, 'with': 0.005781155641966035, 'was': 0.0057590620535254, 'he': 0.005744332994564977, 'dont': 0.0056265005228815935, 'all': 0.

####Bigram

In [0]:
#Bigram with NLTK

bigram_dict = {}

total_bigrams = 0

for sentence in training_data:
  words = nltk.word_tokenize(sentence)
  bigram_list = ngrams(words, 2)

  for bigram_combination in bigram_list:
    total_bigrams += 1
    if bigram_combination in bigram_dict:
      bigram_dict[bigram_combination] += 1
    else:
      bigram_dict[bigram_combination] = 1


bigram_prob_dict = {}

for key, val in reversed(sorted(bigram_dict.items(), key = operator.itemgetter(1))):
  bigram_prob_dict[str(key)] = val / total_bigrams

print ("Probability values bigram, sorted desc order")
print(bigram_prob_dict)


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:15: DeprecationWarning: generator 'ngrams' raised StopIteration
  from ipykernel import kernelapp as app


Probability values bigram, sorted desc order
{"('going', 'to')": 0.012341348820516167, "('you', 'know')": 0.005522203008254755, "('we', 'have')": 0.004510750758589187, "('were', 'going')": 0.004274201442135143, "('and', 'i')": 0.00391529903096349, "('to', 'be')": 0.003907142157982316, "('of', 'the')": 0.0038255734281705763, "('want', 'to')": 0.0032790629384319226, "('have', 'to')": 0.0030425136219778787, "('to', 'do')": 0.0026428268459003554, "('in', 'the')": 0.0025857287350321383, "('i', 'dont')": 0.0025775718620509643, "('a', 'lot')": 0.002479689386276877, "('i', 'think')": 0.0022757675617475284, "('have', 'a')": 0.0021534144670299193, "('i', 'mean')": 0.0021452575940487453, "('and', 'they')": 0.002055531991255832, "('i', 'have')": 0.001990277007406441, "('i', 'said')": 0.001941335769519397, "('lot', 'of')": 0.0018679239126888317, "('and', 'we')": 0.0017129433260465268, "('and', 'you')": 0.0016232177232536134, "('by', 'the')": 0.0015090215015171784, "('to', 'have')": 0.00147639400959

####Trigram

In [0]:
#Trigram with NLTK

trigram_dict = {}

total_trigrams = 0

for sentence in training_data:
  words = nltk.word_tokenize(sentence)
  trigram_list = ngrams(words, 3)

  for trigram_combination in trigram_list:
    total_trigrams += 1
    if trigram_combination in trigram_dict:
      trigram_dict[trigram_combination] += 1
    else:
      trigram_dict[trigram_combination] = 1


trigram_prob_dict = {}

for key, val in reversed(sorted(trigram_dict.items(), key = operator.itemgetter(1))):
  trigram_prob_dict[str(key)] = val / total_trigrams

print ("Probability values trigram, sorted desc order")
#printing first 100 
print(list(trigram_prob_dict.items())[:100])


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:10: DeprecationWarning: generator 'ngrams' raised StopIteration
  # Remove the CWD from sys.path while we load stuff.


Probability values trigram, sorted desc order
[("('were', 'going', 'to')", 0.004664085629697108), ("('going', 'to', 'be')", 0.0024140286950580735), ("('a', 'lot', 'of')", 0.002076975631974493), ("('not', 'going', 'to')", 0.0015850603507173765), ("('we', 'have', 'to')", 0.001503074470507857), ("('going', 'to', 'have')", 0.0011386927806877705), ("('going', 'to', 'do')", 0.0011204736961967661), ("('by', 'the', 'way')", 0.001111364153951264), ("('one', 'of', 'the')", 0.001065816442723753), ("('im', 'going', 'to')", 0.00101115918925074), ("('i', 'want', 'to')", 0.0009838305625142338), ("('the', 'united', 'states')", 0.0009018446823047142), ("('are', 'going', 'to')", 0.0008654065133227055), ("('i', 'dont', 'know')", 0.000838077886586199), ("('and', 'you', 'know')", 0.000838077886586199), ("('youre', 'going', 'to')", 0.0008289683443406969), ("('theyre', 'going', 'to')", 0.0008198588020951948), ("('its', 'going', 'to')", 0.0008016397176041904), ("('and', 'were', 'going')", 0.000774311090867683

####Quadgram

In [0]:
#Quadgram 
quadgram_dict = {}
total_quadgrams = 0

for sentence in training_data:
  words = nltk.word_tokenize(sentence)
  quadgram_list = ngrams(words, 4)
  for quadgram_combination in quadgram_list:
    total_quadgrams += 1
    if quadgram_combination in quadgram_dict:
      quadgram_dict[quadgram_combination] += 1
    else:
      quadgram_dict[quadgram_combination] = 1


quadgram_prob_dict = {}

for key, val in reversed(sorted(quadgram_dict.items(), key = operator.itemgetter(1))):
  quadgram_prob_dict[str(key)] = val / total_quadgrams

print ("Probability values quadgram, sorted desc order") 
print(list(quadgram_prob_dict.items())[:100])


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:10: DeprecationWarning: generator 'ngrams' raised StopIteration
  # Remove the CWD from sys.path while we load stuff.


Probability values quadgram, sorted desc order
[("('and', 'were', 'going', 'to')", 0.0008613087791973423), ("('were', 'going', 'to', 'do')", 0.0006049668806267047), ("('were', 'going', 'to', 'have')", 0.0005844595287410537), ("('its', 'going', 'to', 'be')", 0.0005742058527982281), ("('we', 'are', 'going', 'to')", 0.0005331911490269261), ("('not', 'going', 'to', 'happen')", 0.0005126837971412752), ("('were', 'going', 'to', 'win')", 0.00048192276931279866), ("('were', 'not', 'going', 'to')", 0.00048192276931279866), ("('and', 'by', 'the', 'way')", 0.00046141541742714766), ("('thank', 'you', 'very', 'much')", 0.00045116174148432213), ("('i', 'dont', 'want', 'to')", 0.00043065438959867113), ("('were', 'going', 'to', 'be')", 0.00043065438959867113), ("('were', 'going', 'to', 'make')", 0.0004204007136558456), ("('make', 'america', 'great', 'again')", 0.0003998933617701946), ("('a', 'lot', 'of', 'people')", 0.0003486249820560671), ("('to', 'take', 'care', 'of')", 0.00032811763017041607), ("('

####Generator Function

In [0]:
def Generator(model_name, sentence_length):
#Generate using unigram
  if model_name == 'unigram':
    multi_list = np.random.multinomial(total_num_of_words//sentence_length, [1/sentence_length]*sentence_length, size=1)
    final_sentence = ''
    idx = 0
    for key, val in reversed(sorted(unigram_prob_dict.items(), key = operator.itemgetter(1))):
      for j in multi_list[0]:
        if idx == j:
          final_sentence += key[2:-2]
          final_sentence += ' '
      idx += 1
    print(final_sentence)


# Generate using Bigram
  if model_name == 'bigram':
    multi_list = np.random.multinomial(total_bigrams//sentence_length, [1/sentence_length]*sentence_length, size=1)
    final_sentence = ''
    idx = 0
    for key, val in reversed(sorted(bigram_prob_dict.items(), key = operator.itemgetter(1))):
      for j in multi_list[0]:
        if idx == j:
          final_sentence += key[2:-2]
          final_sentence += ' '

      idx += 1 
    print(final_sentence)

  if model_name == 'trigram':
    multi_list = np.random.multinomial(total_trigrams//sentence_length, [1/sentence_length]*sentence_length, size=1)
    final_sentence = ''
    idx = 0
    for key, val in reversed(sorted(trigram_prob_dict.items(), key = operator.itemgetter(1))):
      for j in multi_list[0]:
        if idx == j:
          final_sentence += key[2:-2]
          final_sentence += ' '
      idx += 1
    print(final_sentence)
    
  if model_name == 'quadgram':
    multi_list = np.random.multinomial(total_quadgrams//sentence_length, [1/sentence_length]*sentence_length, size=1)
    final_sentence = ''
    idx = 0
    for key, val in reversed(sorted(quadgram_prob_dict.items(), key = operator.itemgetter(1))):
      for j in multi_list[0]:
        if idx == j:
          final_sentence += key[2:-2]
          final_sentence += ' '
      idx += 1
    print(final_sentence)    

####Some generated sentences

In [0]:
print(Generator('unigram', 8))

stai  rect opp ra erywhe  e 
None


In [0]:
print(Generator('bigram', 8))

secretary', 'of how', 'we had', 'an to', 'live are', 'saying about', 'trade make', 'sure my', 'family 
None


In [0]:
print(Generator('trigram', 8))

have', 'a', 'strong love', 'you', 'too dont', 'think', 'hes to', 'come', 'through get', 'an', 'environmental billions', 'and', 'billions looking', 'at', 'the the', 'amount', 'of 
None


In [0]:
print(Generator('quadgram', 8))

is', 'a', 'nice', 'guy i', 'see', 'these', 'ads we', 'dont', 'have', 'to in', 'terms', 'of', 'the if', 'we', 'get', 'attacked an', 'environmental', 'impact', 'statement going', 'to', 'do', 'well we', 'can', 'not', 'continue 
None


### Ans 2 (neural based) - explanation inline comments

####Setup for Neural Code

In [0]:
from keras.models import Sequential
from keras.layers import Dense, Activation, SimpleRNN, LSTM, TimeDistributed

all_chars = list(set(speeches))

len_of_vocabulary = len(all_chars)
len_of_sequence = 70

index_to_char = {index:char for index, char in enumerate(all_chars)}
char_to_index = {char:index for index, char in enumerate(all_chars)}


X = np.zeros((len(speeches)//len_of_sequence, len_of_sequence, len_of_vocabulary))
Y = np.zeros((len(speeches)//len_of_sequence, len_of_sequence, len_of_vocabulary))

for i in range(0, len(speeches)//len_of_sequence):
    X_sequence = speeches[i*len_of_sequence:(i+1)*len_of_sequence]
    X_sequence_index = [char_to_index[value] for value in X_sequence]
    input_sequence = np.zeros((len_of_sequence, len_of_vocabulary))
    for j in range(len_of_sequence):
        input_sequence[j][X_sequence_index[j]] = 1.
    X[i] = input_sequence

    Y_sequence = speeches[i*len_of_sequence+1:(i+1)*len_of_sequence+1]
    Y_sequence_index = [char_to_index[value] for value in Y_sequence]
    output_sequence = np.zeros((len_of_sequence, len_of_vocabulary))
    for j in range(len_of_sequence):
        output_sequence[j][Y_sequence_index[j]] = 1.
    Y[i] = output_sequence



def generate_text(model, length):
    ix = [np.random.randint(len_of_vocabulary)]
    y_char = [ix_to_char[ix[-1]]]
    X = np.zeros((1, length, len_of_vocabulary))
    for i in range(length):
        X[0, i, :][ix[-1]] = 1
        print(ix_to_char[ix[-1]], end="")
        ix = np.argmax(model.predict(X[:, :i+1, :])[0], 1)
        y_char.append(ix_to_char[ix[-1]])
    return ('').join(y_char)

In [0]:
model = Sequential()
num_of_dimensions = 400
RNN = SimpleRNN(num_of_dimensions, input_shape=(None, len_of_vocabulary), return_sequences=True)
model.add(RNN)
densed = Dense(len_of_vocabulary, name='dense')
model.add(densed)
model.add(Activation('softmax'))
print(model.summary())
model.compile(loss="categorical_crossentropy", optimizer="rmsprop")

Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
simple_rnn_5 (SimpleRNN)     (None, None, 400)         197200    
_________________________________________________________________
dense (Dense)                (None, None, 92)          36892     
_________________________________________________________________
activation_7 (Activation)    (None, None, 92)          0         
Total params: 234,092
Trainable params: 234,092
Non-trainable params: 0
_________________________________________________________________
None


In [0]:
#Generate Text Vanilla RNN

nb_epoch = 0

iteration = 8
while iteration > 0:
    print('\n')
    iteration -= 1
    model.fit(X, Y, batch_size = 200, verbose = 1, nb_epoch = 1)
    nb_epoch += 1
    generate_text(model, 80)
    if nb_epoch % 10 == 0:
        model.save_weights('checkpoint_{}_epoch_{}.hdf5'.format(8, nb_epoch))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  


Epoch 1/1
 2200/12803 [====>.........................] - ETA: 33s - loss: 3.6851

####LSTM

In [0]:
model = Sequential()

# Any arbitrary value


model.add(LSTM(1200, input_shape=(None, len_of_vocabulary), return_sequences=True))
model.add(LSTM(1200, return_sequences=True))
densed = Dense(len_of_vocabulary, name='dense')
model.add(densed)
model.add(TimeDistributed(Dense(len_of_vocabulary)))
model.add(Activation('softmax'))
print(model.summary())
model.compile(loss="categorical_crossentropy", optimizer="rmsprop")

In [0]:
nb_epoch = 0
iteration = 8 
while iteration > 0:
    print('\n')
    iteration -= 1
    model.fit(X, Y, batch_size = 200, verbose = 1, nb_epoch = 1)
    nb_epoch += 1
    generate_text(model, 80)
model.save('model.h5')